<a href="https://colab.research.google.com/github/hope04302/freeSearch/blob/main/mainCode/hate_detection_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 혐오 발언 탐지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 필요 라이브러리 설치

In [2]:
!pip install transformers
!pip install datasets
!pip install torchmetrics
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 6.5 MB/s eta 0:00:00


## 데이터셋 로드

- 학습, 검증, 테스트 데이터셋 준비
- 라벨 정보

      class_label:
        names:
          0: origin
          1: physical
          2: politics
          3: profanity
          4: age
          5: gender
          6: race
          7: religion
          8: not_hate_speech

In [3]:
from datasets import load_dataset

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

In [4]:
# 데이터 예제 출력

print(train)
print(validation)
print(test)
print(train['text'][0])
print(train['label'][0])

Dataset({
    features: ['text', 'label'],
    num_rows: 78977
})
Dataset({
    features: ['text', 'label'],
    num_rows: 8776
})
Dataset({
    features: ['text', 'label'],
    num_rows: 21939
})
"자한당틀딱들.. 악플질 고만해라."
[2, 4]


In [5]:
# 사전
print(train['text'][0:10])

# 리스트
print(train[0:10])


['"자한당틀딱들.. 악플질 고만해라."', '정치적으로 편향된 평론한은 분은 별로...', '적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...', '"안서는 아재들 풀발기 ㅋㄲㅋ"', '우와 ㅋ 능력자', '맛녀석 콩트보다 약했음맛녀석 애청자로써 70%실력발휘', '주영훈 솔직히 호감임 잉꼬부부로 소문났잖아', '이게주간아이돌이랑머가달라...', '아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사람 짜증나게하냐 외국서 편히살려면 아닥하고 살아라 대한민국서 취미로 돈벌어가지말고 좀 끄지라고!', '"문재인 하는게 뭐 별거있냐?ㅂㅅㅅㅋ가 하는짓인데 어련하겠어.ㅋㅋㅋ"']
{'text': ['"자한당틀딱들.. 악플질 고만해라."', '정치적으로 편향된 평론한은 분은 별로...', '적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...', '"안서는 아재들 풀발기 ㅋㄲㅋ"', '우와 ㅋ 능력자', '맛녀석 콩트보다 약했음맛녀석 애청자로써 70%실력발휘', '주영훈 솔직히 호감임 잉꼬부부로 소문났잖아', '이게주간아이돌이랑머가달라...', '아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사람 짜증나게하냐 외국서 편히살려면 아닥하고 살아라 대한민국서 취미로 돈벌어가지말고 좀 끄지라고!', '"문재인 하는게 뭐 별거있냐?ㅂㅅㅅㅋ가 하는짓인데 어련하겠어.ㅋㅋㅋ"'], 'label': [[2, 4], [8], [2], [4], [8], [8], [8], [8], [3], [2, 3]]}


## 모델 및 토크나이저 로드

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

name_card = "bert-base-multilingual-cased"
num_labels = 9
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(name_card, do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained(name_card, num_labels=num_labels, problem_type="multi_label_classification")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [7]:
# 토크나이저 예시

ids = tokenizer.encode(train['text'][0])
tokenized_words = tokenizer.convert_ids_to_tokens(ids)
model_input = tokenizer(train['text'][0])

print(tokenized_words)
print(ids)
print(model_input)

['[CLS]', '"', '자', '##한', '##당', '##틀', '##딱', '##들', '.', '.', '악', '##플', '##질', '고', '##만', '##해', '##라', '.', '"', '[SEP]']
[101, 107, 9651, 11102, 21928, 119373, 118826, 27023, 119, 119, 9520, 119412, 48599, 8888, 19105, 14523, 17342, 119, 107, 102]
{'input_ids': [101, 107, 9651, 11102, 21928, 119373, 118826, 27023, 119, 119, 9520, 119412, 48599, 8888, 19105, 14523, 17342, 119, 107, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
# 토크나이저 예시2

model_input = tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)
reverse = tokenizer.convert_ids_to_tokens(model_input['input_ids'])
print(reverse)

{'input_ids': [101, 107, 9651, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}
['[CLS]', '"', '자', '[SEP]']


In [9]:
# 토크나이저 예시3

print(model_input)

another_name_card = 'roberta-base'
another_tokenizer = AutoTokenizer.from_pretrained(another_name_card, do_lower_case=False)
model_input = another_tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)

another_name_card = 'distilbert-base-uncased'
another_tokenizer = AutoTokenizer.from_pretrained(another_name_card, do_lower_case=False)
model_input = another_tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)


{'input_ids': [101, 107, 9651, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}


{'input_ids': [0, 113, 43998, 2], 'attention_mask': [1, 1, 1, 1]}


{'input_ids': [101, 1000, 100, 102], 'attention_mask': [1, 1, 1, 1]}


In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

def tokenize_function(examples):
    model_input = tokenizer(examples['text'], max_length=128, truncation=True, padding="max_length")

    mlb = MultiLabelBinarizer(classes=[0,1,2,3,4,5,6,7,8])
    one_hot_labels = mlb.fit_transform(examples['label'])

    model_input['label'] = one_hot_labels

    return model_input

In [11]:
print(train)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_valid = validation.map(tokenize_function, batched=True)

print(tokenized_train)

Dataset({
    features: ['text', 'label'],
    num_rows: 78977
})


Map:   0%|          | 0/78977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8776 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 78977
})


## 모델 학습

In [12]:
from transformers import TrainingArguments

BATCH_SIZE = 64
EPOCHS = 1

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Colab Notebooks/freeSearch2/model',
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    run_name="v1",
    seed=42,
)

In [13]:
import torch
from torchmetrics import Accuracy, F1Score, HammingDistance, AUROC

def compute_metrics(eval_pred):
    threshold = 0.5

    logits, labels = eval_pred
    logits = torch.Tensor(logits)
    labels = torch.Tensor(labels).long()

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(logits))

    preds = torch.zeros(size = probs.size())
    preds[probs >= threshold] = 1

    accuracy = Accuracy(task='multilabel', num_labels=9)
    f1_macro = F1Score(task="multilabel", num_labels=9, average='macro')
    f1_micro = F1Score(task="multilabel", num_labels=9, average='micro')
    f1_weight = F1Score(task="multilabel", num_labels=9, average='weighted')
    auroc = AUROC(task='multilabel', num_labels=9, average='micro')
    hamming = HammingDistance(task="multiclass", num_classes=2)


    metrics = {'accuracy': accuracy(preds, labels),
               'f1_macro': f1_macro(preds, labels),
               'f1_micro': f1_micro(preds, labels),
               'f1_weighted': f1_weight(preds, labels),
               'auroc': auroc(preds, labels),
               'hamming_loss': hamming(preds, labels)}
    return metrics

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
)


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Auroc,Runtime,Samples Per Second,Steps Per Second
1,0.187000,0.053201,0.143228,0.946799,0.635491,0.777200,0.769223,0.857591,67.339100,130.325000,2.049000


TrainOutput(global_step=1235, training_loss=0.1870005866293965, metrics={'train_runtime': 1719.3392, 'train_samples_per_second': 45.935, 'train_steps_per_second': 0.718, 'total_flos': 5195256957082368.0, 'train_loss': 0.1870005866293965, 'epoch': 1.0})

In [16]:
trainer.save_model()

## 예측

In [17]:
tokenized_test = test.map(tokenize_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Colab Notebooks/freeSearch2/model'
)
model.to(device)

Map:   0%|          | 0/21939 [00:00<?, ? examples/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.evaluate()

{'eval_loss': 0.14501458406448364,
 'eval_accuracy': 0.9459359645843506,
 'eval_f1_macro': 0.6368470788002014,
 'eval_f1_micro': 0.7753435373306274,
 'eval_f1_weighted': 0.7683694362640381,
 'eval_auroc': 0.8561519384384155,
 'eval_hamming_loss': 0.054064035415649414,
 'eval_runtime': 169.255,
 'eval_samples_per_second': 129.621,
 'eval_steps_per_second': 2.027}